# Dataloading 01

In this notebook, we'll figure out how to use PyTorch's DataLoader class to load our massive files without reading the entirety of them into memory

In [55]:
import comet_ml
import dask.dataframe as dd
import pandas as pd 
import torch
import linecache 
import csv
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn.functional as F

torch.manual_seed(0)

In [57]:
class LightningNN(pl.LightningModule):
    def __init__(self, N_features, N_labels):
        super(LightningNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(N_features, 512),
            nn.ReLU(),
            *layers,
            nn.Linear(64, N_labels),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", val_loss, on_step=True, on_epoch=True, logger=True)
        return val_loss
    
model = LightningNN(3,3)

We'll first design a custom dataset to use with PyTorch's `DataLoader` class

In [58]:
class GeneExpressionData(Dataset):
    def __init__(self, filename, labelname):
        self._filename = filename
        self._labelname = labelname
        self._total_data = 0
        
        with open(filename, "r") as f:
            self._total_data = len(f.readlines()) - 1
    
    def __getitem__(self, idx):        
        line = linecache.getline(self._filename, idx + 2)
        label = linecache.getline(self._labelname, idx + 2)
        
        csv_data = csv.reader([line])
        csv_label = csv.reader([label])
        
        data = [x for x in csv_data][0]
        label = [x for x in csv_label][0]
        return torch.from_numpy(np.array([float(x) for x in data])).float(), [int(float(x)) for x in label][0]
    
    def __len__(self):
        return self._total_data
    
    def num_labels(self):
        return pd.read_csv(self._labelname)['# label'].nunique()
    
    def num_features(self):
        return len(self.__getitem__(0)[0])

Since PyTorch loss functions require classes in $[0, C]$, we'll first add $1$ to the labels and re-write it out so we can use it for training

In [59]:
def fix_labels(file):
    labels = pd.read_csv(file)
    labels['# label'] = labels['# label'].astype(int) + 1
    labels.to_csv('fixed_' + file.split('/')[-1], index=False)

fix_labels('../data/processed/labels/primary_labels_neighbors_50_components_50_clust_size_100.csv')

Great, we now continue as normal

In [60]:
t = GeneExpressionData(
    filename='../data/processed/umap/primary_reduction_neighbors_100_components_3.csv',
    labelname='fixed_primary_labels_neighbors_50_components_50_clust_size_100.csv'
)

Let's see how fast it takes to load a minibatch of data

In [61]:
%%time 

for i in range(64):
    t.__getitem__(i)

CPU times: user 3.88 ms, sys: 2.64 ms, total: 6.52 ms
Wall time: 5.17 ms


Before we train our model, we need to split our data into training and testing sets, in order to get an unbiased evaluation of our model's performance. Likely, we will initially overfit the training set since we provide no regularization.

In [62]:
train_size = int(0.8 * len(t))
test_size = len(t) - train_size

train, test = torch.utils.data.random_split(t, [train_size, test_size])

In [63]:
traindata = DataLoader(train, batch_size = 8, num_workers = 0)
valdata = DataLoader(test, batch_size = 8, num_workers = 0)

Now that we've defined our `DataLoader`, let's test it when training a simple Neural Network

## Using PyTorch Lightning

PyTorch lightning seems nicer than Ignite, especially for GPU training. Let's test it out 

In [70]:
class NN(pl.LightningModule):
    def __init__(self, N_features, N_labels, weights):
        super(NN, self).__init__()
        self.weights = weights
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(N_features, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            *layers,
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, N_labels),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
    def accuracy(self, y_hat, y):
        y_hat = torch.argmax(y_hat, dim=1)
        accuracy = torch.sum(y == y_hat).item() / (len(y) * 1.0)
        return torch.tensor(accuracy)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y, weight=self.weights)
        acc = self.accuracy(y_hat, y)

        self.log("train_loss", loss, on_step=True, on_epoch=True, logger=True)
        self.log("train_accuracy", acc, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y, weight=self.weights)
        acc = self.accuracy(y_hat, y)

        self.log("val_loss", val_loss, on_step=True, on_epoch=True, logger=True)
        self.log("train_accuracy", acc, on_step=True, on_epoch=True, logger=True)
        return val_loss
    

In [71]:
from sklearn.utils.class_weight import compute_class_weight

def class_weights(label_df):
    label_df = pd.read_csv(label_df)
    
    weights = compute_class_weight(
        class_weight='balanced', 
        classes=np.unique(label_df), 
        y=label_df.values.reshape(-1)
    ) 

    weights = torch.from_numpy(weights)

    return weights.float()

weights = class_weights('fixed_primary_labels_neighbors_50_components_50_clust_size_100.csv')

In [72]:
weights

tensor([  0.6428,   2.0281,  43.2046,   1.8374,   0.8580,  33.3467, 102.0523,
        100.3226,   0.5029,   0.6207,   2.0534,   0.4522,   0.3983,  13.3462,
          1.3319,   0.3946])

In [73]:
model = NN(t.num_features(), t.num_labels(), weights)

In [74]:
comet_logger = pl.loggers.CometLogger(
        api_key="neMNyjJuhw25ao48JEWlJpKRR",
        project_name="gene-expression-classification",  # Optional
        experiment_name="test2"
    )


trainer = pl.Trainer(auto_lr_find=True, max_epochs=10, logger=comet_logger)
trainer.fit(model, traindata, valdata)


CometLogger will be initialized in online mode
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/jlehrer1/gene-expression-classification/ac4395be63614a59a6841c183dd3364c
COMET INFO:   Others:
COMET INFO:     Name : test2
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (8.18 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     model graph              : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jup

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [75]:
trainer.callback_metrics

{'train_loss': tensor(2.5939),
 'train_loss_step': tensor(2.5939),
 'train_accuracy': tensor(0.),
 'train_accuracy_step': tensor(0.)}